In [ ]:
%pip install datasets
%pip install huggingface
%pip install peft
%pip install transformers
%pip install trl
%pip install bitsandbytes
%pip install einops
%pip install scipy

In [ ]:
import os
from dataclasses import dataclass, field
from typing import Optional

import torch
from datasets import load_dataset
from datasets import load_from_disk
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
)
from tqdm.notebook import tqdm

from trl import SFTTrainer
from huggingface_hub import interpreter_login

interpreter_login()



In [ ]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype='float16',
    bnb_4bit_use_double_quant=False,
)
device_map = {"": 0}

In [ ]:

#Download model
model = AutoModelForCausalLM.from_pretrained(
        "microsoft/phi-2", 
        quantization_config=bnb_config, 
        device_map=device_map,
        trust_remote_code=True,
        use_auth_token=True
    )

# Disable gradient checkpointing
model.config.gradient_checkpointing = False


peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=32,
    target_modules=['lm_head.linear', 'transformer.embd.wte'],
    bias="none",
    task_type="CAUSAL_LM", 
)


In [ ]:
def format_conversations(example):
    formatted_text = ""
    for conversation in example['conversations']:
        if conversation['from'] == 'human':
            formatted_text += "Instruct: " + conversation['value'] + "\\n"
        elif conversation['from'] == 'gpt':
            formatted_text += "Output: " + conversation['value'] + "\\n"
    return {'text': formatted_text.replace('\n','\\n')}



In [ ]:

tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

training_arguments = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    save_steps=500, 
    logging_steps=10,
    learning_rate=2e-4,
    fp16=False,
    bf16=True,
    max_grad_norm=.3,
    max_steps=10000,
    warmup_ratio=.03,
    group_by_length=True,
    lr_scheduler_type="constant",
)

model.config.use_cache = False

dataset = load_dataset("json", data_files="instruct.json", split="train")
dataset = dataset.map(format_conversations)


In [ ]:

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=2048,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)

trainer.train()

In [ ]:
model.save_pretrained('./ftphi2')
tokenizer.save_pretrained('./ftphi2')


In [ ]:
from huggingface_hub import Repository

repo = Repository(local_dir='./ftphi2', clone_from='******')
repo.git_add('*')
repo.git_commit('Add model')
repo.git_push()
